 **Aplicación de Machine Learning al análisis de sentimientos:**

# 0.Contexto inicial

Esta rutina se basa en el capítulo 8 del libro *Machine Learning with PyTorch and Scikit-Learn* de Raschka. En el cual se pretende profundizar en un subcampo del Procesamiento de Lenguaje Natural (NLP), llamado **Análisis de Sentimientos**.

"El análisis de sentimiento se refiere al uso de procesamiento de lenguaje natural, análisis de texto y lingüística computacional para identificar y extraer información subjetiva de los recursos."

El propósito específico es clasificar documentos a partir de los sentimientos sugeridos por la actitud del escritor.


En este ejercicio se usará un dataset compuesto por 50.000 reseñas cinematográficas extraídas de la plataforma *Internet Movie Database* (IMDb) para construir un predictor que pueda distinguir entre una reseña positiva y una negativa. Es decir, es un problema de clasificación, por ende, es aprendizaje supervisado, así que se esperan los datos etiquetados.




---



En el libro se definen las siguientes etapas para el proyecto:


1.   Limpieza y preparación de datos de texto
2.   Construcción de 'feature vectors' (vector de características)
3. Entrenamiento del modelo para clasificar las reseñas

4. Inferencia de temas a partir de colecciones de documentos para su categorización



# 1. Limpieza y preparación de los datos

Lo primero será importar las librerías necesarias para el proyecto, con las versiones que se trabajan en el libro.



## Importar librerías

### Instalación y verificación de versiones

In [24]:
!pip install pyprind==2.11.3

In [ ]:
import numpy as np
import pandas as pd
import sklearn
import pyprind
import nltk

# Verificar versiones instaladas
def check_version(library, required_version):
    try:
        current_version = library.__version__
        if current_version == required_version:
            print(f'{library.__name__} está instalado y tiene la versión {current_version}.')
        else:
            print(f'{library.__name__} está instalado, pero la versión {current_version} no cumple con los requisitos. Se requiere la versión {required_version}.')
    except AttributeError:
        print(f'{library.__name__} no está instalado.')

# Verificar e instalar numpy
check_version(np, '1.21.2')
if '1.21.2' not in np.__version__:
    !pip install numpy==1.21.2

# Verificar e instalar pandas
check_version(pd, '1.3.2')
#if '1.3.2' not in pd.__version__:
    #!pip install pandas==1.3.2

# Verificar e instalar sklearn
check_version(sklearn, '1.0')
if '1.0' not in sklearn.__version__:
    !pip install scikit-learn==1.0

# Verificar e instalar pyprind
check_version(pyprind, '2.11.3')
if '2.11.3' not in pyprind.__version__:
    !pip install pyprind==2.11.3

# Verificar e instalar nltk
check_version(nltk, '3.6')
if '3.6' not in nltk.__version__:
    !pip install nltk==3.6


### Importación directa

In [25]:
#!pip install pyprind==2.11.3
import numpy as np
import pandas as pd
import sklearn
import pyprind
import nltk

## Importar datos

Primero se hará subiendo la base de datos localmente

In [ ]:
df = pd.read_csv('/content/movie_data.csv', encoding='utf-8')
## El parámetro encoding en Python se utiliza para especificar
##la codificación de caracteres al leer o escribir archivos de texto.
##Indica cómo deben interpretarse los bytes para convertirlos
## en caracteres y viceversa.

### UTF-8: 'utf-8' (predeterminado)
### Codificación de ancho variable que puede representar cada carácter
### en el conjunto de caracteres.

# the following is necessary on some computers:
df = df.rename(columns={"0": "review", "1": "sentiment"})

df.head(5)

Alternativamente se usará la versión publicada en la web desde un drive

In [26]:
url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vSoYnlADbf3wGATt9Nt7r28tq5ugshktK8a1VxFbNORbh1EJoDptdsluV0f9QSNc8R7TPz26CWDPI5k/pub?output=csv'
df_2 = pd.read_csv(url, encoding='utf-8')
#df_2.head(5)

df = pd.read_csv(url, encoding='utf-8')
df.head(5)

,review,sentiment
0,"In 1974, the teenager Martha Moxley (Maggie Gr...",1
1,OK... so... I really like Kris Kristofferson a...,0
2,"***SPOILER*** Do not read this, if you think a...",0
3,hi for all the people who have seen this wonde...,1
4,"I recently bought the DVD, forgetting just how...",0


Para entender mejor los datos se debe mencionar que el etiquetado se hace según el siguiente criterio:



*   **Positivo (1):** si la película fue calificada con 6 o más estrellas en IMDb
*   **Negativo (0):** si la película fue calificada con 5 o menos estrellas en IMDb


---


Ya que por comodidad se trabajará con un archivo *CSV* previamente procesado y reindexado aleatoriamente, el autor recomienda como **buena práctica** revisar la dimensión del DataFrame.


In [28]:
print(df.shape)
print(df_2.shape)

(50000, 2)
(50000, 2)


En ambos casos se cuentan con las 50.000 samples u observaciones y las únicas dos columnas.

# 2. Transformando palabras a **feature vectors**

## 2.1 Modelo **"Bag-of-words"** (Bolsa de Palabras)

Dentro del procesamiento del conjunto de datos se debe convertir la data categórica, como el texto o palabras, en formato numérico antes de aplicar algún algoritmo de ML.

En este caso, se usará el modelo "Bag-of-words", cuya idea subyacente es la siguiente:


1.   Creamos un vocabulario de tokens únicos -por ejemplo, palabras- a partir de todo el conjunto de documentos.
2.   Construimos un vector de características (feature vector) a partir de cada documento que contiene los recuentos de la frecuencia con la que cada palabra aparece en el documento en cuestión.

---

Acá se debe notar que como las palabras en cada documento representan un pequeño subconjunto de toda la **Bolsa de palabras**, la mayoría de entradas del **vector de características** serán nulas, por esto es que podríamos denominarlos vectores **dispersos** (*sparse*).



---





Para construir la Bolsa de palabras usaremos la clase **CountVectorizer** implementada en scikit-learn.

Como se verá, esta clase toma un array de datos de texto, cuyas entradas pueden ser párrafos, textos u oraciones y construye el modelo de Bolsa de palabras respectivo.

In [31]:
from sklearn.feature_extraction.text import CountVectorizer
count = CountVectorizer()
docs = np.array(['El sol brilla',
                 'El clima es agradable',
                 'El sol brilla, el clima es agradable',
                 'y uno y uno son dos'])

bag = count.fit_transform(docs)


In [30]:
#CountVectorizer?

Aplicando aquí el método(función) .fit_transform se construye el vocabulario
del modelo de bolsa de palabras y transformó esas oraciones en features vectors.

Ahora veamos el vocabulario (transcripción) que se realizó:

In [32]:
count.vocabulary_ ## Nótese que esto es equivalente a llamar el método .vocabulary  de la clase CountVectorizer()

{'el': 4,
 'sol': 6,
 'brilla': 1,
 'clima': 2,
 'es': 5,
 'agradable': 0,
 'uno': 8,
 'son': 7,
 'dos': 3}

Como lo acabamos de ver el vocabulario es almacenado como un diccionario de Python que asigna un entero a cada palabra inédita.

Ahora veamos el vector de características creado:

In [33]:
bag.toarray()

array([[0, 1, 0, 0, 1, 0, 1, 0, 0],
       [1, 0, 1, 0, 1, 1, 0, 0, 0],
       [1, 1, 1, 0, 2, 1, 1, 0, 0],
       [0, 0, 0, 1, 0, 0, 0, 1, 2]])

In [34]:
bag.toarray()[1].shape

(9,)

En estos vectores cada índice mostrado corresponde al entero que se almacenó en el diccionario de palabras (en el CountVectorizer vocabulary).

Se pueden hacer las siguientes observaciones:



1.   Usualmente los enteros se asignan en orden alfabético según las palabras de la Bolsa.
2.   La dimensión de los vectores será $m$x$1$, donde m es el mayor de los enteros asignados en la Bolsa.
3. Las entradas de dichos vectores de características se denominan las **"Frecuencias Brutas de los términos"**
(raw term frequencies) y se representan por $tf(t,d)$.

Las frecuencias Brutas de los términos ($tf(t,d)$) miden el número de veces que un término t aparece en un documento(oración) d. Además, no importa el orden en el que aparezcan dichos términos (palabras).



---




**Modelos de N-gramas:**

La secuencia de elementos del modelo de bolsa de palabras que acabamos de crear también se denomina modelo de 1-grama o unigrama.

1-grama o unigrama: cada elemento o token del vocabulario representa una sola palabra.

La clase CountVectorizer en scikit-learn nos permite utilizar diferentes modelos de n-gramas a través de su parámetro ngram_range. Aunque por defecto se utiliza una representación de 1 gramo, podríamos cambiar a una representación de 2gramas inicializando una nueva instancia de CountVectorizer con ngram_range=(2,2).



---



In [ ]:
#CountVectorizer?  ##Se puede llamar la ayuda en cualquier momento para más informacióm

## 2.2 Valoración de la relevancia de las palabras mediante la frecuencia de términos y la frecuencia inversa de documentos

Cuando estamos analizando datos de texto, a menudo nos encontramos con palabras que aparecen en varios documentos de ambas clases (reseñas positivas y negativas).

Estas palabras que ocurren con frecuencia generalmente no contienen información útil o discriminatoria. En esta subsección, aprenderemos acerca de una técnica útil llamada frecuencia de término-frecuencia inversa de documento (tf-idf) que se puede utilizar para ponderar a la baja esas palabras que ocurren con frecuencia en los vectores de características.

 El tf-idf se puede definir como el producto de la frecuencia del término y la frecuencia inversa del documento:

 $$\text{tf-idf}(t,d)=\text{tf (t,d)}\times \text{idf}(t,d)$$

 Aquí, tf(t, d) es la frecuencia del término que presentamos en la sección anterior, y la frecuencia inversa del documento idf(t, d) se puede calcular como:

 $$\text{idf}(t,d) = \text{log}\frac{n_d}{1+\text{df}(d, t)},$$

 donde $n_d$ es el número total de documentos y df(d, t) es el número de documentos d que contienen el término t.

 Nótese que agregar la constante 1 al denominador es opcional y sirve para asignar un valor no nulo a los términos que aparecen en todos los ejemplos de entrenamiento; el logaritmo se utiliza para asegurar que las frecuencias bajas de algunos documentos no se ponderen demasiado.

---

 Scikit-learn implementa otro transformador, el `TfidfTransformer`, que toma las frecuencias de términos crudos de `CountVectorizer` como entrada y las transforma en tf-idfs:



In [35]:
from sklearn.feature_extraction.text import TfidfTransformer

tfidf = TfidfTransformer(use_idf=True, ### Activar la reponderación de la frecuencia inversa del documento. Si es False, idf(t) = 1.
                         norm='l2', ## Se usa la norma l2 o la euclidiana, entre las opciones está l1(taxista) y l2
                         smooth_idf=True) ### Esto incluye sumar un 1 en el numerador y denominador anteriores

print(tfidf.fit_transform(count.fit_transform(docs))
      .toarray())

[[0.         0.61366674 0.         0.         0.49681612 0.
  0.61366674 0.         0.        ]
 [0.52303503 0.         0.52303503 0.         0.42344193 0.52303503
  0.         0.         0.        ]
 [0.36222092 0.36222092 0.36222092 0.         0.5864981  0.36222092
  0.36222092 0.         0.        ]
 [0.         0.         0.         0.40824829 0.         0.
  0.         0.40824829 0.81649658]]


In [ ]:
#TfidfTransformer?

Las ecuaciones para el idf y el tf-idf que se implementaron en scikit-learn son:

$$\text{idf} (t,d) = log\frac{1 + n_d}{1 + \text{df}(d, t)}$$

$$\text{tf-idf}(t,d) = \text{tf}(t,d) \times (\text{idf}(t,d)+1)$$

Aunque es más común normalizar las frecuencias brutas de los términos antes de calcular los tf-idfs, el `TfidfTransformer` normaliza los tf-idfs directamente.

Por defecto (`norm='l2'`), el TfidfTransformer de scikit-learn aplica la normalización L2, que devuelve un vector de longitud 1 dividiendo un vector de características no normalizado *v* por su norma L2:

$$v_{\text{norm}} = \frac{v}{||v||_2} = \frac{v}{\sqrt{v_{1}^{2} + v_{2}^{2} + \dots + v_{n}^{2}}} = \frac{v}{\big (\sum_{i=1}^{n} v_{i}^{2}\big)^\frac{1}{2}}$$




Veamos un ejemplo:

In [36]:
count_2 = CountVectorizer()
docs_2 = np.array([
        'The sun is shining',
        'The weather is sweet',
        'The sun is shining, the weather is sweet, and one and one is two'])
bag_2 = count_2.fit_transform(docs_2)


In [37]:
count_2.vocabulary_

{'the': 6,
 'sun': 4,
 'is': 1,
 'shining': 3,
 'weather': 8,
 'sweet': 5,
 'and': 0,
 'one': 2,
 'two': 7}

In [38]:
bag_2.toarray()

array([[0, 1, 0, 1, 1, 0, 1, 0, 0],
       [0, 1, 0, 0, 0, 1, 1, 0, 1],
       [2, 3, 2, 1, 1, 1, 2, 1, 1]])

La palabra "is" tiene una frecuencia de término de 3 (tf = 3) en el documento 3 ($d_3$), y la frecuencia de documento de este término es 3 ya que el término "is" aparece en los tres documentos (df = 3). Por lo tanto, podemos calcular el idf de la siguiente manera:

$$\text{idf}("is", d_3) = log \frac{1+3}{1+3} = 0$$

Ahora, para calcular el tf-idf, basta con sumar 1 a la frecuencia inversa del documento y multiplicarlo por la frecuencia del término:
$$\text{tf-idf}("is", d_3)= 3 \times (0+1) = 3$$

In [39]:
tf_is = 3
n_docs = 3
def tfidf(tf, n_docs):
  idf = np.log((n_docs+1) / (3+1))
  tfidf = tf_is * (idf + 1)
  return tfidf

tfidf_is = tfidf(tf_is, n_docs)
print(f'La tf-idf del término (palabra) "is" es: {tfidf_is:.2f}')

La tf-idf del término (palabra) "is" es: 3.00


##2.3 Adecuación datos de texto

El primer paso importante -antes de construir nuestro modelo de bolsa de palabras- es limpiar los datos de texto
eliminando todos los caracteres no deseados. Por ejemplo:

In [40]:
df.loc[0, 'review'][-50:]

'is seven.<br /><br />Title (Brazil): Not Available'

Como se pudo ver el texto contiene marcas HTML, así como signos de puntuación y otros caracteres no alfabéticos.

Aunque las marcas HTML no contienen mucha semántica útil, los signos de puntuación pueden representar información adicional útil en determinados contextos de PNL.

Sin embargo, para simplificar, vamos a
eliminar todos los signos de puntuación excepto los emoticones, como :), ya que son útiles para el análisis de sentimientos.

---
Para realizar esta tarea, utilizaremos la biblioteca de expresiones regulares (regex) de Python, re, como se muestra aquí:

In [42]:
import re
def preprocessor(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)',
                           text)
    text = (re.sub('[\W]+', ' ', text.lower()) + ### El text.lower() convierte en minúscula todo el texto
            ' '.join(emoticons).replace('-', ''))
    return text

In [43]:
#re.findall?

In [44]:
preprocessor(df.loc[0, 'review'][-50:])

'is seven title brazil not available'

In [45]:
preprocessor("</a>This :) is :( a test :-)!")

'this is a test :) :( :)'

Ahora se aplicará a todo el conjunto de datos

In [46]:
df['review'] = df['review'].apply(preprocessor)

## 2.4 Procesamiento de documentos en tokens

Después de preparar con éxito el conjunto de datos de reseñas de películas, ahora debemos pensar en cómo dividir el corpus de texto en elementos individuales. Una forma de tokenizar documentos es dividirlos en palabras individuales al separar los documentos limpios en sus caracteres de espacio en blanco:

In [47]:
def tokenizer(text):
    return text.split()

tokenizer('runners like running and thus they run')

['runners', 'like', 'running', 'and', 'thus', 'they', 'run']

Otra técnica útil es la derivación de palabras (word stemming), que es el proceso de transformar una palabra en su forma raíz. Esto nos permite asignar palabras relacionadas a la misma raíz. El algoritmo original de derivación de palabras fue desarrollado por Martin F. Porter en 1979 y, por lo tanto, se conoce como el algoritmo de derivación de Porter

In [50]:
from nltk.stem.porter import PorterStemmer

porter = PorterStemmer()
def tokenizer_porter(text):
    return [porter.stem(word) for word in text.split()]

print(tokenizer_porter('runners like running and thus they run'))
print(tokenizer_porter('en españoles también funciona felicidades'))

['runner', 'like', 'run', 'and', 'thu', 'they', 'run']
['en', 'español', 'también', 'funciona', 'felicidad']


### 2.4.1 Eliminación de palabras vacías (stop words)

Las palabras vacías son simplemente aquellas palabras que son extremadamente comunes en todo tipo de textos y probablemente no contienen (o contienen muy poca) información útil que pueda usarse para distinguir entre diferentes clases de documentos. Ejemplos de palabras vacías son "is", "and", "has", y "like".


---


Para eliminar las palabras vacías de las reseñas de películas, utilizaremos el conjunto de 127 palabras vacías en inglés que está disponible en la biblioteca NLTK, y que se puede obtener llamando a la función `nltk.download`:

In [51]:
import nltk

nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [52]:
from nltk.corpus import stopwords

stop = stopwords.words('english')
[w for w in tokenizer_porter('a runner likes running and runs a lot')
 if w not in stop]

['runner', 'like', 'run', 'run', 'lot']

# 3. Entrenamiento de un modelo de regresión logística para clasificar las reseñas

entrenaremos un modelo de regresión logística para clasificar las reseñas de películas en reseñas positivas y negativas basándonos en el modelo de bolsa de palabras. Primero, dividiremos el DataFrame de documentos de texto limpio en 25,000 documentos para entrenamiento (train) y 25,000 documentos para prueba (test):

In [ ]:
X_train = df.loc[:25000, 'review'].values
y_train = df.loc[:25000, 'sentiment'].values
X_test = df.loc[25000:, 'review'].values
y_test = df.loc[25000:, 'sentiment'].values

In [ ]:
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV

tfidf = TfidfVectorizer(strip_accents=None,
                        lowercase=False,
                        preprocessor=None)

"""
param_grid = [{'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              {'vect__ngram_range': [(1, 1)],
               'vect__stop_words': [stop, None],
               'vect__tokenizer': [tokenizer, tokenizer_porter],
               'vect__use_idf':[False],
               'vect__norm':[None],
               'clf__penalty': ['l1', 'l2'],
               'clf__C': [1.0, 10.0, 100.0]},
              ]
"""

small_param_grid = [{'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [None],
                     'vect__tokenizer': [tokenizer, tokenizer_porter],
                     'clf__penalty': ['l2'],
                     'clf__C': [1.0, 10.0]},
                    {'vect__ngram_range': [(1, 1)],
                     'vect__stop_words': [stop, None],
                     'vect__tokenizer': [tokenizer],
                     'vect__use_idf':[False],
                     'vect__norm':[None],
                     'clf__penalty': ['l2'],
                  'clf__C': [1.0, 10.0]},
              ]

lr_tfidf = Pipeline([('vect', tfidf),
                     ('clf', LogisticRegression(solver='liblinear'))])

gs_lr_tfidf = GridSearchCV(lr_tfidf, small_param_grid,
                           scoring='accuracy',
                           cv=5,
                           verbose=1,
                           n_jobs=-1)

 Para el clasificador de regresión logística, estamos utilizando el solver LIBLINEAR, ya que puede funcionar mejor que la opción predeterminada ('lbfgs') para conjuntos de datos relativamente grandes.

Nota importante sobre n_jobs

Ten en cuenta que se recomienda encarecidamente usar n_jobs=-1 (en lugar de n_jobs=1) en el ejemplo de código anterior para utilizar todos los núcleos disponibles en tu máquina y acelerar la búsqueda en cuadrícula.


---



In [ ]:
gs_lr_tfidf.fit(X_train, y_train)

Fitting 5 folds for each of 8 candidates, totalling 40 fits


/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:528: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


GridSearchCV(cv=5,
             estimator=Pipeline(steps=[('vect',
                                        TfidfVectorizer(lowercase=False)),
                                       ('clf',
                                        LogisticRegression(solver='liblinear'))]),
             n_jobs=-1,
             param_grid=[{'clf__C': [1.0, 10.0], 'clf__penalty': ['l2'],
                          'vect__ngram_range': [(1, 1)],
                          'vect__stop_words': [None],
                          'vect__tokenizer': [<function tokenizer at 0x7e57faa54af0>,
                                              <function tokenizer_porter at 0x7e57faaa55a0...
                          'vect__stop_words': [['i', 'me', 'my', 'myself', 'we',
                                                'our', 'ours', 'ourselves',
                                                'you', "you're", "you've",
                                                "you'll", "you'd", 'your',
                                                'yours', 'yourself',
                                                'yourselves', 'he', 'him',
                                                'his', 'himself', 'she',
                                                "she's", 'her', 'hers',
                                                'herself', 'it', "it's", 'its',
                                                'itself', ...],
                                               None],
                          'vect__tokenizer': [<function tokenizer at 0x7e57faa54af0>],
                          'vect__use_idf': [False]}],
             scoring='accuracy', verbose=1)

Cuando inicializamos el objeto GridSearchCV y su cuadrícula de parámetros utilizando el código anterior, nos limitamos a una cantidad limitada de combinaciones de parámetros, ya que la cantidad de vectores de características, así como el gran vocabulario, pueden hacer que la búsqueda en cuadrícula sea computacionalmente costosa. Utilizando una computadora de escritorio estándar, nuestra búsqueda en cuadrícula puede llevar de 5 a 10 minutos para completarse.

En el ejemplo de código anterior, reemplazamos CountVectorizer y TfidfTransformer de la subsección anterior con TfidfVectorizer, que combina CountVectorizer con TfidfTransformer. Nuestra param_grid consistió en dos diccionarios de parámetros. En el primer diccionario, usamos TfidfVectorizer con su configuración predeterminada (use_idf=True, smooth_idf=True y norm='l2') para calcular los tf-idfs; en el segundo diccionario, configuramos esos parámetros a use_idf=False, smooth_idf=False y norm=None para entrenar un modelo basado en frecuencias de términos crudas. Además, para el clasificador de regresión logística en sí, entrenamos modelos usando regularización L2 a través del parámetro de penalización y comparamos diferentes fuerzas de regularización definiendo un rango de valores para el parámetro de inversa de regularización C.

In [ ]:
print(f'Best parameter set: {gs_lr_tfidf.best_params_}')
print(f'CV Accuracy: {gs_lr_tfidf.best_score_:.3f}')

Best parameter set: {'clf__C': 10.0, 'clf__penalty': 'l2', 'vect__ngram_range': (1, 1), 'vect__stop_words': None, 'vect__tokenizer': <function tokenizer at 0x7e57faa54af0>}
CV Accuracy: 0.897


In [ ]:
clf = gs_lr_tfidf.best_estimator_
print(f'Test Accuracy: {clf.score(X_test, y_test):.3f}')

Test Accuracy: 0.899


# 4. Asignación latente de Dirichlet (LDA)

LDA es un modelo probabilístico generativo que intenta encontrar grupos de palabras que aparecen con frecuencia juntas en diferentes documentos. Estas palabras que aparecen con frecuencia representan nuestros temas, asumiendo que cada documento es una mezcla de diferentes palabras. La entrada a un modelo LDA es el modelo de bolsa de palabras

Dado una matriz de bolsa de palabras como entrada, LDA la descompone en dos nuevas matrices:


*   Una matriz de documentos a temas.
*   Una matriz de palabras a temas.

LDA descompone la matriz de bolsa de palabras de tal manera que si multiplicamos esas dos matrices, podremos reproducir la entrada, la matriz de bolsa de palabras, con el error más bajo posible.

En la práctica, estamos interesados en los temas que LDA encontró en la matriz de bolsa de palabras. La única desventaja puede ser que debemos definir el número de temas de antemano; el número de temas es un hiperparámetro de LDA que debe especificarse manualmente.

## LDA usando scikit-learn

utilizaremos la clase LatentDirichletAllocation implementada en scikit-learn para descomponer el conjunto de datos de reseñas de películas y categorizarlo en diferentes temas. En el siguiente ejemplo, restringiremos el análisis a 10 temas diferentes

In [20]:
from sklearn.feature_extraction.text import CountVectorizer

count = CountVectorizer(stop_words='english',
                        max_df=.1,  #establecimos la frecuencia máxima de documentos de las palabras a considerar en un 10 por ciento
                        max_features=5000) #limitamos el número de palabras a considerar a las 5,000 palabras más frecuentes para limitar la dimensionalidad y mejorar las iinferencias
X = count.fit_transform(df['review'].values) ## explicar el .values

In [11]:
type(df['review'].values)

numpy.ndarray

In [21]:
from sklearn.decomposition import LatentDirichletAllocation

lda = LatentDirichletAllocation(n_components=10, ### número de temas
                                random_state=123,
                                learning_method='batch') ## puede ser 'batch' u 'online', donde batch es aprendizaje por lotes, se entrena con todos los datos disponibles y no se actualiza
X_topics = lda.fit_transform(X)

In [12]:
lda.components_.shape

(10, 5000)

Se debe tener en cuenta que los valores de importancia de las palabras están clasificados en orden creciente. Por lo tanto, para imprimir las cinco palabras principales, necesitamos ordenar el array de temas en orden inverso.

In [22]:
n_top_words = 5
feature_names = count.get_feature_names_out()

for topic_idx, topic in enumerate(lda.components_):
    print(f'Topic {(topic_idx + 1)}:')
    print(' '.join([feature_names[i]
                    for i in topic.argsort()\
                        [:-n_top_words - 1:-1]]))

Topic 1:
horror effects gore budget killer
Topic 2:
worst minutes guy money script
Topic 3:
war american book series history
Topic 4:
performance beautiful performances excellent feel
Topic 5:
comedy kids tv family series
Topic 6:
woman wife father town men
Topic 7:
space fi sci effects earth
Topic 8:
music song songs musical dance
Topic 9:
role version plays original performance
Topic 10:
action game fight animation series


In [23]:
horror = X_topics[:,0 ].argsort()[::-1]

for iter_idx, movie_idx in enumerate(horror[:5]):
    print(f'\nPelícula horror #{(iter_idx + 1)}:')
    print(df['review'][movie_idx][:300], '...')


Película horror #1:
over christmas break a group of college friends stay behind to help prepare the dorms to be torn down and replaced by apartment buildings to make the work a bit more difficult a murderous chucks wearing psycho is wandering the halls of the dorm preying on the group in various violent ways registered ...

Película horror #2:
twelve years ago production stopped on the slasher flick hot blooded since almost everyone on the set started dying now a couple of film students have decided to finish the film despite the fact that there s a rumor that the film is cursed well they re about to find out that some curses are real whe ...

Película horror #3:
i am an avid fan of lucio fulci and yet i must say that zombi 3 aka zombie flesh eaters 2 of 1988 which he made with two other directors bruno mattei and claudi fragasso was quite a disappointment especially compared to its great predecessor fulci s very own gore classic zombi 2 aka zombie felsh eat ...

Película horror #4:
f